<a href="https://colab.research.google.com/github/lexie-z/Introduction-to-ML-Fall2021/blob/main/Final_assignment_Yiqing_Zhu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read data

In [ ]:
import pandas as pd

filepath = '/content/drive/My Drive/9796 final/LLM_ECSignals_2010-2024_v5_complete_remapped3_FullCounts_finalFormat.csv'

# Initialize an empty list to hold the filtered chunks
filtered_chunks = []

# Read the CSV in chunks
for chunk in pd.read_csv(filepath, chunksize=10000, sep=',', index_col=False, dtype='unicode'):  # Adjust chunksize based on your system's memory
    # only need the document score from the total document section
    filtered_chunk = chunk[chunk['SignalType'] == 'Total']
    filtered_chunks.append(filtered_chunk)

# Concatenate all the filtered chunks into a single DataFrame
data = pd.concat(filtered_chunks)

# Apply Ratio and Exp_weight scoring methods

In [ ]:
import numpy as np
data['DocDate'] = pd.to_datetime(data['DocDate'])
data.sort_values(by=['DocDate'], inplace=True)
data.set_index('DocDate', inplace=True)

def ratio_exp_score(data,method):
  # since the data for each company may not be available each quater
  # we calculate the a rolling average score for the last 4 quarters using time window
  data['RollingAvg_'+method] = data.groupby('Ticker')['EventsScore_'+method].rolling(window='365D').mean().values
  data['Ratio_'+method] = (data['EventsScore_'+method].astype(float) - data['RollingAvg_'+method]) / (1 - data['RollingAvg_'+method])
  data['Exp_Weight_'+method] = (data['EventsScore_'+method].astype(float) - data['RollingAvg_'+method]) * np.exp(2 * data['RollingAvg_'+method])
  return data

methods = ['4_2_1','1_1_0','1_1_1','3_1_0']
for method in methods:
  data_bt = ratio_exp_score(data,method)
data_bt.tocsv('/content/drive/My Drive/9796 final/backtest_signal.csv') # save for convenience

# Adjust dates and resampling
smooth the portfolio by carrying forward the score & push all reporting dates to the 1st of the month to avoid look-ahead bias

In [122]:
import pandas as pd
data_bt = pd.read_csv('/content/drive/My Drive/9796 final/backtest_signal.csv')
data_bt.drop(columns=['Unnamed: 0','EX_NAME','QTR_YEAR'],inplace=True)
data_bt['DocDate'] = pd.to_datetime(data_bt['DocDate'])
# push forward reporting dtaes
# if a company reported on January 20th, no action is taken until February 1st using the close price
def adjust_date(date):
    if date.day > 1:
        return (date + pd.DateOffset(months=1)).replace(day=1)
    return date
data_bt['Adjusted DocDate'] = data_bt['DocDate'].apply(adjust_date)
data_bt.drop(columns=['DocDate'],inplace=True)
data_bt.drop_duplicates(subset=['Adjusted DocDate','Ticker'],keep='last',inplace=True)
data_bt.set_index('Adjusted DocDate', inplace=True)
# resample data to monthly frequency
# carry forward the relative score, always taking the most recent available
grouped_resampled_df = data_bt.groupby('Ticker').resample('MS').ffill()

# Drop the extra 'Ticker' level from the index after grouping
grouped_resampled_df.reset_index(level=0, drop=True, inplace=True)

In [123]:
grouped_resampled_df

,Ticker,COMPANYID,BESTTICKER,EXCHANGE,EX_CODE,EventsScore_1_1_1,Ratio_1_1_1,Exp_Weight_1_1_1,EventsScore_4_2_1,Ratio_4_2_1,Exp_Weight_4_2_1,EventsScore_1_1_0,Ratio_1_1_0,Exp_Weight_1_1_0,EventsScore_3_1_0,Ratio_3_1_0,Exp_Weight_3_1_0
Adjusted DocDate,,,,,,,,,,,,,,,,,
2016-12-01,10004497|TPB,10004497,TPB,NYSE,US,0.642857,-0.505393,-0.551245,0.619159,-0.560636,-0.620513,0.662577,-0.395680,-0.435843,0.599190,-0.602546,-0.675252
2017-01-01,10004497|TPB,10004497,TPB,NYSE,US,0.642857,-0.505393,-0.551245,0.619159,-0.560636,-0.620513,0.662577,-0.395680,-0.435843,0.599190,-0.602546,-0.675252
2017-02-01,10004497|TPB,10004497,TPB,NYSE,US,0.642857,-0.505393,-0.551245,0.619159,-0.560636,-0.620513,0.662577,-0.395680,-0.435843,0.599190,-0.602546,-0.675252
2017-03-01,10004497|TPB,10004497,TPB,NYSE,US,0.642857,-0.505393,-0.551245,0.619159,-0.560636,-0.620513,0.662577,-0.395680,-0.435843,0.599190,-0.602546,-0.675252
2017-04-01,10004497|TPB,10004497,TPB,NYSE,US,0.642857,-0.505393,-0.551245,0.619159,-0.560636,-0.620513,0.662577,-0.395680,-0.435843,0.599190,-0.602546,-0.675252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-01,99942756|TTD,99942756,TTD,NasdaqGM,US,0.784158,-0.196313,-0.182438,0.766326,-0.170115,-0.168366,0.779874,-0.183114,-0.173520,0.753585,-0.156045,-0.160468
2023-12-01,99942756|TTD,99942756,TTD,NasdaqGM,US,0.834879,0.718347,0.963375,0.843348,0.741234,0.987974,0.826693,0.692489,0.934197,0.847674,0.753337,0.999664
2024-01-01,99942756|TTD,99942756,TTD,NasdaqGM,US,0.834879,0.718347,0.963375,0.843348,0.741234,0.987974,0.826693,0.692489,0.934197,0.847674,0.753337,0.999664


# Calculate trading positions based on signals
 (1) Examine rolling mean of the scores in month (t), the previous (t-1), and the next previous (t-2) for all companies to assign the ranking cut-offs for the current month.

 (2) Determine the long cutoffs for scores ranked in the top 95%, and the short cutoffs for scores ranked in the bottom 5%. Then we use 1 in position variable to represent long position, and use -1 in position variable to represent short position.

 (3) Resample positions to daily data by carrying forward the trading decision made on the 1st of each month so that positions align with daily return data.

In [124]:
def position_pipeline(signal, grouped_resampled_df, strategy):
    # Ensure data is sorted by index
    grouped_resampled_df.sort_index(inplace=True)

    # Calculate the rolling mean of the signal across a 3-month window for each ticker
    # To assign the ranking cut-offs for the current month, we examine the scores of all companies in month (t), the previous (t-1), and the next previous (t-2).
    rolling_scores = pd.DataFrame(grouped_resampled_df.groupby('Ticker')[signal].rolling(window=3, min_periods=1).mean())
    rolling_scores.reset_index(inplace=True)

    # Determine long cutoffs for scores ranked in the top 95%
    long_cutoffs = rolling_scores.groupby('Adjusted DocDate')[signal].apply(lambda x: np.percentile(x, 95))

    longs = pd.DataFrame({'Long Cutoff': long_cutoffs})

    # Initialize positions DataFrame from the original data
    positions = grouped_resampled_df[['Ticker', signal, 'BESTTICKER']].copy()
    positions['Long Position'] = 0

    # Assign positions for long strategy
    longs.sort_index(inplace=True)
    for date in longs.index:
        long_cutoff = longs.loc[date, 'Long Cutoff']
        positions.loc[positions.index == date, 'Long Position'] = (positions.loc[positions.index == date, signal] >= long_cutoff).astype(int)

    # Check if strategy is long-short, then compute short positions
    if strategy == 'long-short':
        short_cutoffs = rolling_scores.groupby('Adjusted DocDate')[signal].apply(lambda x: np.percentile(x, 5))
        shorts = pd.DataFrame({'Short Cutoff': short_cutoffs})
        positions['Short Position'] = 0
        shorts.sort_index(inplace=True)
        for date in shorts.index:
            short_cutoff = shorts.loc[date, 'Short Cutoff']
            positions.loc[positions.index == date, 'Short Position'] = (positions.loc[positions.index == date, signal] <= short_cutoff).astype(int) * (-1)

        # Combine long and short positions to get the final position
        positions['Position'] = positions['Long Position'] + positions['Short Position']
    else:
        # In long-only strategy, the position is simply the long position
        positions['Position'] = positions['Long Position']

    # Filter out cases where BESTTICKER is null, which also implies a null Ticker
    positions = positions[positions['BESTTICKER'].notnull()]

    # Resample positions from monthly to daily data
    #positions.reset_index(inplace=True)
    #positions.set_index(['Adjusted DocDate', 'Ticker'], inplace=True)
    #positions = positions[['Position']]
    # Forward fill positions daily within each ticker
    # If a company remains in the top/bottom, the open position is simply extended.
    # When a company falls out of the threshold range, the position is closed.
    #positions = positions.groupby('Ticker').apply(lambda x: x.droplevel('Ticker').resample('D').ffill())

    return positions


# Extract Close prices for all tickers in yahoo finance
 (1) Use BESTTICKER as ticker identifier. This is consistent with the ticker name that can be extracted from Ticker.
 (2)


In [46]:
import yfinance as yf

# Use BESTTICKER as ticker identifier
tickers = grouped_resampled_df['BESTTICKER'].unique().tolist()
start_date = grouped_resampled_df.index.min().strftime('%Y-%m-%d')
end_date = grouped_resampled_df.index.max().strftime('%Y-%m-%d')

close_prices = yf.download(tickers, start=start_date, end=end_date, interval='1mo')['Close']
close_prices

[                       0%%                      ]  10 of 14441 completed

$RALY: possibly delisted; No price data found  (1mo 2010-02-01 -> 2024-03-01)
$GMCR: possibly delisted; No price data found  (1mo 2010-02-01 -> 2024-03-01)


[                       0%%                      ]  21 of 14441 completed

$PVA: possibly delisted; No price data found  (1mo 2010-02-01 -> 2024-03-01)
$TBRA: possibly delisted; No price data found  (1mo 2010-02-01 -> 2024-03-01)
$DANG: possibly delisted; No price data found  (1mo 2010-02-01 -> 2024-03-01)


[                       0%%                      ]  27 of 14441 completed

...

[*********************100%%**********************]  14430 of 14441 completed

$VITA: possibly delisted; No price data found  (1mo 2010-02-01 -> 2024-03-01)
$EPL: possibly delisted; No price data found  (1mo 2010-02-01 -> 2024-03-01)


[*********************100%%**********************]  14441 of 14441 completed
ERROR:yfinance:
9378 Failed downloads:
ERROR:yfinance:['MEKO', 'PCTN', 'TA', 'RVR', '3AMLJ', 'NBPE', 'ACLGATI', '1828', 'GLG', 'BSIF', 'CU.', '3197', 'CLI', 'MRU.A', 'WAND', 'RRI', 'RKLY', 'MP1', 'AESE', '4776', 'ANIM', 'A035760', 'JMBDA', 'SDR', 'NMCI', 'LIZI', 'CRTX.1', 'BRPR3', 'QGMD', 'A323410', 'STRI', 'HLC.', 'TMPO', 'SPG.', 'GCHE', 'SALT B', 'COLBUN', 'GOODLUCK', 'DLS.', '8550B', 'MITO', 'LOGG3', 'BWCU', 'UU.', 'INNT', 'COUP', 'PRFO', 'AMCN', 'TPZ.', 'V2RETAIL', 'YHOO', 'HPT', 'CBTX', 'SBIO.', 'PTBL3', 'TAHO', 'PITTIENG', 'ERA', 'CEQP', 'RTP', 'GRND3', 'ALR', 'BPE.', 'BARBEQUE', 'BMTC', 'NATIONALUM', 'DANSKE', 'COLPAL', 'PKI.U', 'CVLB', 'MGROS', 'BRP.2', 'RTIX', 'ACKB', 'FLMN', 'YLO.', 'IEC', 'GDP', 'ASR.', '3GNTA', 'CPG.', 'AZRN', 'TLRD', 'APCOTEXIND', 'WABAG', 'SESGL', '1688', '2772', 'HDI.', 'KELLTONTEC', 'TPEIR', 'SANGHIIND', 'VIC.', '2533', '4262', 'MGMBA', 'CLEAN', 'APPH', 'NTRI', 'USIMINAS', 'TRM

$APWR: possibly delisted; No price data found  (1mo 2010-02-01 -> 2024-03-01)


Ticker,000001,000002,000381532,0004B,000914210,001253020,002202,0030B,0033A,003742,...,ZX,ZXAIY,ZY,ZYDUSLIFE,ZYDUSWELL,ZYME,ZYNE,ZYXI,ZZ,ZZZ.
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.809091,NaN,NaN
2010-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.827273,NaN,NaN
2010-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.863636,NaN,NaN
2010-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.563636,NaN,NaN
2010-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.527273,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.02,NaN,8.880000,NaN,NaN
2023-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,8.87,NaN,9.160000,NaN,NaN
2023-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.39,NaN,10.890000,NaN,NaN


In [58]:
# double check tickers in close_prices match the bestticker in positions
cnt = 0
tickers_cp = close_prices.columns.tolist()
for ticker in tickers_cp:
  if ticker in tickers:
    cnt += 1
print('Tickers in close_prices match tickers in positions:',cnt==close_prices.shape[1])

Tickers in close_prices match tickers in positions: True


In [60]:
# drop tickers that are delisted and have no price data in close_prices
close_prices.dropna(axis=1,how='all',inplace=True)
close_prices

Ticker,A,AA,AACG,AADI,AAL,AAMC,AAN,AAOI,AAON,AAP,...,ZS,ZTO,ZTS,ZUMZ,ZUO,ZVIA,ZVRA,ZWS,ZYME,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-01,22.503576,31.959900,3.80,NaN,7.33,NaN,NaN,NaN,4.156049,40.799999,...,NaN,NaN,NaN,14.340000,NaN,NaN,NaN,NaN,NaN,0.809091
2010-03-01,24.599428,34.218719,4.03,NaN,7.35,NaN,NaN,NaN,4.468148,41.919998,...,NaN,NaN,NaN,20.490000,NaN,NaN,NaN,NaN,NaN,0.827273
2010-04-01,25.937054,32.272289,3.53,NaN,7.07,NaN,NaN,NaN,4.770370,45.099998,...,NaN,NaN,NaN,18.559999,NaN,NaN,NaN,NaN,NaN,0.863636
2010-05-01,23.147352,27.970921,3.25,NaN,8.83,NaN,NaN,NaN,4.875062,51.759998,...,NaN,NaN,NaN,17.120001,NaN,NaN,NaN,NaN,NaN,0.563636
2010-06-01,20.336195,24.174179,3.00,NaN,8.61,NaN,NaN,NaN,4.604444,50.180000,...,NaN,NaN,NaN,16.110001,NaN,NaN,NaN,NaN,NaN,0.527273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-01,103.370003,25.639999,0.97,4.38,11.15,3.605882,7.41,7.43,54.480000,52.029999,...,158.690002,23.570000,157.000000,16.430000,7.41,1.94,4.48,26.459999,7.02,8.880000
2023-11-01,127.800003,26.860001,0.88,5.26,12.43,3.260000,8.81,13.25,62.599998,50.790001,...,197.529999,22.320000,176.669998,18.879999,9.12,2.15,4.76,29.440001,8.87,9.160000
2023-12-01,139.029999,34.000000,1.16,2.02,13.74,4.140000,10.88,19.32,73.870003,61.029999,...,221.559998,21.280001,197.369995,20.340000,9.40,2.01,6.55,29.410000,10.39,10.890000


In [125]:
close_prices = pd.read_csv('/content/drive/My Drive/9796 final/close_prices.csv')
close_prices.set_index('Date',inplace=True)
returns = (close_prices - close_prices.shift(1))/close_prices.shift(1)
returns.index = pd.to_datetime(returns.index)

In [126]:
returns

,A,AA,AACG,AADI,AAL,AAMC,AAN,AAOI,AAON,AAP,...,ZS,ZTO,ZTS,ZUMZ,ZUO,ZVIA,ZVRA,ZWS,ZYME,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-01,0.093134,0.070677,0.060526,NaN,0.002729,NaN,NaN,NaN,0.075095,0.027451,...,NaN,NaN,NaN,0.428870,NaN,NaN,NaN,NaN,NaN,0.022472
2010-04-01,0.054376,-0.056882,-0.124070,NaN,-0.038095,NaN,NaN,NaN,0.067639,0.075859,...,NaN,NaN,NaN,-0.094192,NaN,NaN,NaN,NaN,NaN,0.043955
2010-05-01,-0.107557,-0.133284,-0.079320,NaN,0.248939,NaN,NaN,NaN,0.021946,0.147672,...,NaN,NaN,NaN,-0.077586,NaN,NaN,NaN,NaN,NaN,-0.347369
2010-06-01,-0.121446,-0.135739,-0.076923,NaN,-0.024915,NaN,NaN,NaN,-0.055511,-0.030525,...,NaN,NaN,NaN,-0.058995,NaN,NaN,NaN,NaN,NaN,-0.064515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-01,-0.075568,-0.117688,-0.133929,-0.095041,-0.129586,-0.111594,-0.292264,-0.322698,-0.042026,-0.069730,...,0.019924,-0.024824,-0.097597,-0.076966,-0.100728,-0.126126,-0.070539,-0.055675,0.107255,0.110000
2023-11-01,0.236335,0.047582,-0.092784,0.200913,0.114798,-0.095922,0.188934,0.783311,0.149046,-0.023832,...,0.244754,-0.053034,0.125287,0.149117,0.230769,0.108247,0.062500,0.112623,0.263533,0.031532
2023-12-01,0.087872,0.265823,0.318182,-0.615970,0.105390,0.269939,0.234960,0.458113,0.180032,0.201614,...,0.121652,-0.046595,0.117168,0.077331,0.030702,-0.065116,0.376050,-0.001019,0.171364,0.188865


There can be duplicate tickers (which we use to match tickers on yahoo finance) on the same docdate. To avoid confusion, we delete duplicated tickers.

In [132]:
def process_positions(positions_df):
    # Identify duplicates (on the same day and same ticker)
    duplicates = positions_df.duplicated(subset=['Adjusted DocDate', 'Ticker'], keep=False)
    positions_df = positions_df[~duplicates]
    # Pivot positions DataFrame after removing duplicates
    positions_pivot = positions_df.pivot(index='Adjusted DocDate', columns='Ticker', values='Position')
    return positions_pivot

reframe position

In [133]:
import matplotlib.pyplot as plt

def calculate_and_plot_pnl(positions_pivot, returns_df, signal, strategy):
    # Align positions with the returns DataFrame
    aligned_positions = positions_pivot.reindex(returns_df.index, method='ffill').fillna(0)
    # Calculate daily PnL only considering days with returns
    # Here we use returns_df * aligned_positions to get the PnL for each ticker each day
    daily_pnl = aligned_positions.mul(returns_df.shift(-1), axis=0)
    average_daily_pnl = daily_pnl.apply(lambda x: x.dropna().mean() if not x.dropna().empty else 0, axis=1)
    # Calculate cumulative PnL
    cumulative_pnl = (average_daily_pnl + 1).cumprod() - 1

    plt.figure(figsize=(10, 6))
    cumulative_pnl.plot()
    plt.title('Cumulative Returns for '+strategy+' strategy using '+signal)
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return')
    plt.grid(True)
    plt.show()


In [134]:
signal_strategy = {'Ratio_1_1_0':'long-short','Exp_Weight_1_1_0':'long-short','Ratio_4_2_1':'long_only','Exp_Weight_4_2_1':'long only'}

for signal, strategy in signal_strategy.items():

  positions_df = position_pipeline(signal, grouped_resampled_df, strategy).reset_index()
  positions_df['Adjusted DocDate'] = pd.to_datetime(positions_df['Adjusted DocDate'])
  # Extract ticker symbols from the 'Ticker' column in positions_df
  positions_df['Ticker'] = positions_df['Ticker'].apply(lambda x: x.split('|')[1])
  positions_pivot = process_positions(positions_df)
  calculate_and_plot_pnl(positions_pivot, returns, signal, strategy)

ValueError: Index contains duplicate entries, cannot reshape

In [130]:
positions_pivot

Ticker,000001,000002,000381532,0004B,000914210,001253020,002202,0030B,0033A,003742,...,ZX,ZXAIY,ZY,ZYDUSLIFE,ZYDUSWELL,ZYME,ZYNE,ZYXI,ZZ,ZZZ.
Adjusted DocDate,,,,,,,,,,,,,,,,,,,,,
2010-02-01,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2010-03-01,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2010-04-01,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2010-05-01,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2010-06-01,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-01,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.0,0.0,NaN,1.0,NaN,0.0
2023-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,1.0,NaN,0.0
2024-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.0,NaN,NaN


In [131]:
positions_df

,Adjusted DocDate,Ticker,Exp_Weight_4_2_1,BESTTICKER,Long Position,Position
0,2010-02-01,OZRK,-0.215541,OZRK,0,0
1,2010-02-01,VLO,-3.456258,VLO,0,0
2,2010-02-01,CBU,-1.326783,CBU,0,0
3,2010-02-01,NVS,-0.286070,NVS,0,0
4,2010-02-01,WCBO,-1.891990,WCBO,0,0
...,...,...,...,...,...,...
1003770,2024-03-01,SLRC,0.880330,SLRC,0,0
1003771,2024-03-01,LMAT,0.726276,LMAT,0,0
1003772,2024-03-01,GRT.UN,-1.835140,GRT.UN,0,0
1003773,2024-03-01,SJ.,0.039591,SJ.,0,0
